### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
#Sklearn
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE

#other
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')


%matplotlib inline

# Plotly 
from plotly import tools
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

#spaCy
import spacy
nlp = spacy.load("fr_core_news_lg")
from spacy.lang.fr.stop_words import STOP_WORDS
from spacy.lang.fr import French
import string
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

### Load (acceptably clean) data

In [2]:
data=pd.read_pickle("../../data/cleaned2.pkl")
data_np=data.to_numpy()

In [3]:
data=np.load('../../data/organizations_col.npy', allow_pickle=True)

In [4]:
data_single=list(dict.fromkeys(data))

In [5]:
data_single=[x for x in data_single if str(x) != 'nan']

In [6]:
len(data_single)

1161

In [7]:
i=random.randint(1, 1161)
data_single[i]

"Commune d'Eguisheim"

In [8]:
import progressbar
bar = progressbar.ProgressBar(maxval=len(data_np), \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])


## Process data to extract keywords only

### Extract the locs 

In [27]:
def extract_locs(doc):
    all_locs=[None]*len(doc)
    bar.start()
    for i in range(len(doc)):
        this_doc=[]
        for token in nlp(doc[i]):
            if token.ent_type_=='LOC':
                this_doc.append(token.text)
        all_locs[i]=this_doc
        bar.update(i+1)
    bar.finish()
    return(all_locs)

import unicodedata
def strip_accents(text):

    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text)

In [28]:
all_locs=extract_locs(data_single)

[========================================================================] 100%


In [31]:
all_locs

[['Paris'],
 ['Chambéry'],
 ['Corse'],
 ['Orléans', 'Métropole'],
 ['Saint-Denis', 'La', 'Réunion'],
 ['Réseau', ' ', 'régional', 'Inspection'],
 [],
 ['Gamarthe'],
 ['Île-de-France'],
 [],
 ['Paris', 'Seine', 'Ouest'],
 [],
 [],
 ['Antibes'],
 [],
 ['Département', 'de', 'Loire', 'Atlantique'],
 ['Région', 'Île-de-France'],
 ['Région', 'Centre', 'Val', 'de', 'Loire'],
 ['Département', 'de', "l'", 'Hérault'],
 ['Haute-Garonne'],
 [],
 ['Eurométropole', 'de', 'Strasbourg'],
 ['Département', 'du', 'Doubs'],
 ['Métropole', 'Nice'],
 [],
 ['Saint-Claude'],
 [],
 ['Nantes', 'Métropole'],
 [],
 [],
 [],
 ['Région', 'Reunion'],
 ['France'],
 ['Française'],
 [],
 ['Syndicat', 'Mixte', 'des', 'Stations', 'du', 'Mercantour'],
 [],
 ['Tours', 'Métropole', 'Val', 'de', 'Loire'],
 ['Ville', 'de', 'Saint-Egrève'],
 ['Région', 'Provence', 'Alpes', 'Côte', 'Azur'],
 [],
 [],
 [],
 [],
 [],
 ['Communauté', "d'", 'agglomération', 'du', 'Val', 'de', 'Fensch'],
 [],
 ['Bourgogne', 'Franche', 'Comté'],
 [],

In [30]:
#np.save('../../data/locs_org_alg0.npy',np.array(all_locs), fix_imports=True, allow_pickle=True)

In [33]:
stop_loc=['Région', 'Métropole', 'Region', 'Metropole','Mer', 'mer', 'Département', 'DEPARTEMENT', 'Agglomération', 'agglomération','Communauté', 'communauté']

In [34]:
all_locs_ns=[None]*len(all_locs)
for i in range(len(all_locs)):
    mini_list=[x for x in all_locs[i] if x not in stop_loc]
    all_locs_ns[i]=mini_list

In [35]:
all_locs_ns_str=all_locs_ns
for i in range(len(all_locs_ns)):
    all_locs_ns_str[i]=' '.join(all_locs_ns[i])
    

In [36]:
len(all_locs_ns_str)

1161

In [37]:
all_locs_np=np.array(all_locs)

In [39]:
np.save('../../data/locs_org_alg0.npy',np.array(all_locs_ns_str), fix_imports=True, allow_pickle=True)

In [29]:
#np.save('../../data/locs_insee_str.npy', all_locs_ns_str, allow_pickle=True)

### Compare with insee tables

In [10]:
all_locs_ns_str=np.load('../../data/locs_insee_str.npy', allow_pickle=True)

In [11]:
all_locs_ns_str[:10]

array(['Paris île-de-france', 'Chambéry', 'chambéry Chambéry', 'corse',
       'corse', 'orléans métropole Orléans', 'pr saint-denis geospatial',
       '', '', 'capvern nan capvern département des hautes-pyrénées'],
      dtype=object)

In [12]:
departements=pd.read_csv('../../data/departement2019.csv')
communes=pd.read_csv('../../data/communes-01012019.csv')
regions=pd.read_csv('../../data/region2019.csv')

In [13]:
communes_names=communes['nccenr'].to_numpy()

In [14]:
departements_names=departements['nccenr'].to_numpy()

In [15]:
regions_names=regions['nccenr'].to_numpy()

In [16]:
not_comm_names=list(departements_names)+list(regions_names)##+['France', 'Europe', 'Monde']

In [17]:
communes_prblm = list(np.load('../../data/communes_prblm.npy', allow_pickle=True))

In [60]:
np.save('../../data/communes_prblm.npy', np.array(communes_prblm, dtype=object), allow_pickle=True)

In [ ]:
communes_prblm=[]
for i in communes_names:
    for j in not_comm_names:
        if fuzz.token_set_ratio(i,j)==100:
            communes_prblm.append(i)
    

In [35]:
all_locs_ns_str[25]

'Saint-Claude'

In [36]:
regions_names

array(['Guadeloupe', 'Martinique', 'Guyane', 'La Réunion', 'Mayotte',
       'Île-de-France', 'Centre-Val de Loire', 'Bourgogne-Franche-Comté',
       'Normandie', 'Hauts-de-France', 'Grand Est', 'Pays de la Loire',
       'Bretagne', 'Nouvelle-Aquitaine', 'Occitanie',
       'Auvergne-Rhône-Alpes', "Provence-Alpes-Côte d'Azur", 'Corse'],
      dtype=object)

In [40]:
from fuzzywuzzy import fuzz
def is_in(locs, category):
    values=[]
    for i in category:
        values.append(fuzz.token_set_ratio(locs, i))
        maxi=max(values)
    max_item=[i for i, j in enumerate(values) if j == maxi]   
   # print(max_item)
    if values[max_item[0]]==100:
        found=True
        if len(max_item)>1:
            values2=[]
            for w in max_item:
                values2.append(fuzz.ratio(locs, category[w]))
          #  print(values2)
            max_item_item=values2.index(max(values2))
            max_item=[max_item[max_item_item]]
    else:
        found=False
    return(max_item[0], found)

In [41]:
def text_to_loc(text):
    if text=='':
        return(None)
    if (text.find(' france')!=-1) or (text.find(' France')!=-1):
            return('France')
    max_item, found_c=is_in(text, communes_names)
    location=communes.loc[[max_item]]
    if communes_names[max_item] in communes_prblm:
        found_c=False
    if found_c==False:
        max_item, found_d=is_in(text, not_comm_names)
        try:
            location=departements.loc[[max_item]]
        except:
            location=regions.loc[[max_item-len(departements_names)]]
        return(location)
    return(location)

In [58]:
location=text_to_loc('Paris île-de-cité france.')

In [59]:
location

'France'

In [19]:
'île de france'.find('  france')

-1

In [42]:
i=random.randint(1,len(all_locs_ns_str))
#i=27944
print('TEXTE INITIAL: '+ data_np[i])
t = time.time()
location=text_to_loc(all_locs_ns_str[i])
elapsed = time.time() - t
print('MOTS EXTRAITS'+ all_locs_ns_str[i])
print(' ')
print('Localisation INSEE:')
print(location)
print('Computed in '+str(elapsed) )

TEXTE INITIAL: tronçon de voie - saint-étienne-de-baïgorry. Saint-Étienne-de-Baïgorry . les voies sont représentées par leur axes et découpées chaque intersection. l’écriture des toponymes d’origine basque été validée par euskaltzaindia académie de la langue basque la mise jour est effectuée après délibération du conseil municipal.. adresse gps localisation voirie
MOTS EXTRAITSde la Charente Maritime
 
Localisation INSEE:
   dep  reg cheflieu  tncc                ncc             nccenr  \
16  17   75    17300     3  CHARENTE MARITIME  Charente-Maritime   

              libelle  
16  Charente-Maritime  
Computed in 6.5447328090667725


### Essais avec une bonne precision donc plus de contrainte

In [42]:
from fuzzywuzzy import fuzz
def is_in(locs, category):
    values=[]
    for i in category:
        values.append(fuzz.partial_ratio(locs, i))
        maxi=max(values)
    max_item=[i for i, j in enumerate(values) if j == maxi]   
   # print(max_item)
    if values[max_item[0]]==100:
        found=True
        if len(max_item)>1:
            values2=[]
            for w in max_item:
                values2.append(fuzz.ratio(locs, category[w]))
          #  print(values2)
            max_item_item=values2.index(max(values2))
            max_item=[max_item[max_item_item]]
    else:
        found=False
    return(max_item[0], found)

In [43]:
def text_to_loc(text):
    if text=='':
        return(None)
    if (text.find(' france')!=-1) or (text.find(' France')!=-1):
            return('France')
    max_item, found_c=is_in(text, communes_names)
    location=communes.loc[[max_item]]
    if communes_names[max_item] in communes_prblm:
        found_c=False
    if found_c==False:
        max_item, found_d=is_in(text, not_comm_names)
        try:
            location=departements.loc[[max_item]]
        except:
            location=regions.loc[[max_item-len(departements_names)]]
        return(location)
    return(location)

In [44]:
i=random.randint(1,len(all_locs_ns_str))
#i=27944
print('TEXTE INITIAL: '+ data_single[i])
t = time.time()
location=text_to_loc(data_single[i])
elapsed = time.time() - t
print('MOTS EXTRAITS'+ data_single[i])
print(' ')
print('Localisation INSEE:')
print(location)
print('Computed in '+str(elapsed) )

TEXTE INITIAL: Parc Naturel Régional du Vercors
MOTS EXTRAITSParc Naturel Régional du Vercors
 
Localisation INSEE:
      typecom    com   reg dep  arr  tncc    ncc nccenr libelle   can  \
14392     COM  35214  53.0  35  351     0  PARCE  Parcé   Parcé  3508   

       comparent  
14392        NaN  
Computed in 6.616219997406006


In [47]:
from tqdm import tqdm
locations_orgs=[]
for i in tqdm(data_single):
    locations_orgs.append(text_to_loc(i))

100%|██████████| 1161/1161 [2:07:57<00:00,  6.61s/it] 


In [48]:
np.save('../../data/locs_org_alg1.npy',np.array(locations_orgs), fix_imports=True, allow_pickle=True)

In [41]:
locations_orgs

[      typecom    com   reg dep  arr  tncc    ncc nccenr libelle  can  \
 31824     COM  75056  11.0  75  751     0  PARIS  Paris   Paris  NaN   
 
        comparent  
 31824        NaN  ,
       typecom    com   reg dep  arr  tncc       ncc    nccenr   libelle   can  \
 31260     COM  73065  84.0  73  732     0  CHAMBERY  Chambéry  Chambéry  7399   
 
        comparent  
 31260        NaN  ,
     reg cheflieu  tncc    ncc nccenr libelle
 17   94    2A004     0  CORSE  Corse   Corse,
       typecom    com   reg dep  arr  tncc      ncc   nccenr  libelle   can  \
 17873     COM  45234  24.0  45  452     1  ORLEANS  Orléans  Orléans  4599   
 
        comparent  
 17873        NaN  ,
     dep  reg cheflieu  tncc         ncc      nccenr     libelle
 99  974    4    97411     0  LA REUNION  La Réunion  La Réunion,
    reg cheflieu  tncc                  ncc               nccenr  \
 6   24    45234     2  CENTRE VAL DE LOIRE  Centre-Val de Loire   
 
                libelle  
 6  Centre-Val 

In [ ]:
#39298 huat de france et français 

In [45]:
all_locs_ns_str[20751]

'rivière marne de châlons-en-champagne Direction Départementale des Territoires de la Marne'

In [224]:
communes_names[31343]#8183

'Porte-de-Savoie'

In [122]:
other=list(regions_names)+list(departements_names)+['France']

In [44]:
communes

,typecom,com,reg,dep,arr,tncc,ncc,nccenr,libelle,can,comparent
0,COM,01001,84.0,01,012,5,ABERGEMENT CLEMENCIAT,Abergement-Clémenciat,L'Abergement-Clémenciat,0108,NaN
1,COM,01002,84.0,01,011,5,ABERGEMENT DE VAREY,Abergement-de-Varey,L'Abergement-de-Varey,0101,NaN
2,COM,01004,84.0,01,011,1,AMBERIEU EN BUGEY,Ambérieu-en-Bugey,Ambérieu-en-Bugey,0101,NaN
3,COM,01005,84.0,01,012,1,AMBERIEUX EN DOMBES,Ambérieux-en-Dombes,Ambérieux-en-Dombes,0122,NaN
4,COM,01006,84.0,01,011,1,AMBLEON,Ambléon,Ambléon,0104,NaN
...,...,...,...,...,...,...,...,...,...,...,...
37925,COM,97613,6.0,976,NaN,0,M TSANGAMOUJI,M'Tsangamouji,M'Tsangamouji,97613,NaN
37926,COM,97614,6.0,976,NaN,1,OUANGANI,Ouangani,Ouangani,97610,NaN
37927,COM,97615,6.0,976,NaN,0,PAMANDZI,Pamandzi,Pamandzi,97611,NaN
37928,COM,97616,6.0,976,NaN,0,SADA,Sada,Sada,97612,NaN


In [144]:
dpt_prblm=[]
for i in departements_names:
    for j in regions_names:
        if fuzz.token_set_ratio(i,j)==100:
            dpt_prblm.append(i)

In [48]:
for i in communes_names:
    if fuzz.token_set_ratio(i,'drôme')==100:
        print(i)

Balleroy-sur-Drôme
Lande-sur-Drôme
Val de Drôme
Livron-sur-Drôme
Loriol-sur-Drôme


In [146]:
regions_names

array(['Guadeloupe', 'Martinique', 'Guyane', 'La Réunion', 'Mayotte',
       'Île-de-France', 'Centre-Val de Loire', 'Bourgogne-Franche-Comté',
       'Normandie', 'Hauts-de-France', 'Grand Est', 'Pays de la Loire',
       'Bretagne', 'Nouvelle-Aquitaine', 'Occitanie',
       'Auvergne-Rhône-Alpes', "Provence-Alpes-Côte d'Azur", 'Corse'],
      dtype=object)

In [164]:
departements_names

array(['Ain', 'Aisne', 'Allier', 'Alpes-de-Haute-Provence',
       'Hautes-Alpes', 'Alpes-Maritimes', 'Ardèche', 'Ardennes', 'Ariège',
       'Aube', 'Aude', 'Aveyron', 'Bouches-du-Rhône', 'Calvados',
       'Cantal', 'Charente', 'Charente-Maritime', 'Cher', 'Corrèze',
       "Côte-d'Or", "Côtes-d'Armor", 'Creuse', 'Dordogne', 'Doubs',
       'Drôme', 'Eure', 'Eure-et-Loir', 'Finistère', 'Corse-du-Sud',
       'Haute-Corse', 'Gard', 'Haute-Garonne', 'Gers', 'Gironde',
       'Hérault', 'Ille-et-Vilaine', 'Indre', 'Indre-et-Loire', 'Isère',
       'Jura', 'Landes', 'Loir-et-Cher', 'Loire', 'Haute-Loire',
       'Loire-Atlantique', 'Loiret', 'Lot', 'Lot-et-Garonne', 'Lozère',
       'Maine-et-Loire', 'Manche', 'Marne', 'Haute-Marne', 'Mayenne',
       'Meurthe-et-Moselle', 'Meuse', 'Morbihan', 'Moselle', 'Nièvre',
       'Nord', 'Oise', 'Orne', 'Pas-de-Calais', 'Puy-de-Dôme',
       'Pyrénées-Atlantiques', 'Hautes-Pyrénées', 'Pyrénées-Orientales',
       'Bas-Rhin', 'Haut-Rhin', 'Rhône',

In [129]:
len(communes_prblm)

809

In [114]:
data_np[3422]

"concentrations en nitrates d'origine agricole - - eaux souterraines - france entière. Système d'Information sur l'Eau. la directive 91 cee du 12 décembre dite directive nitrates vise protéger la qualité de eau en prévenant la pollution des eaux souterraines et superficielles par les nitrates d’origine agricole notamment en promouvant usage des bonnes pratiques agricoles de gestion de l’azote. ses orientations sont largement reprises dans la directive cadre sur l’eau dce 60 ce du 23 octobre . elle impose aux états membres de : •\tréaliser des campagnes de surveillance des concentrations en nitrates dans les milieux aquatiques eaux superficielles et souterraines au moins tous les quatre ans •\tdésigner des zones de protection spécifiques atteintes ou menacées par la pollution par les nitrates d’origine agricole dites zones vulnérables •\télaborer des programmes d’actions dont les mesures doivent être obligatoirement appliquées par les agriculteurs qui exercent leurs activités en zones v

### Pandas concatenation and stuff

In [19]:
regions.loc[[10]]

,reg,cheflieu,tncc,ncc,nccenr,libelle
10,44,67482,2,GRAND EST,Grand Est,Grand Est


In [20]:
departements.loc[[3]]

,dep,reg,cheflieu,tncc,ncc,nccenr,libelle
3,04,93,04070,4,ALPES DE HAUTE PROVENCE,Alpes-de-Haute-Provence,Alpes-de-Haute-Provence


In [29]:
communes.loc[[7563]]

,typecom,com,reg,dep,arr,tncc,ncc,nccenr,libelle,can,comparent
7563,COM,21407,27.0,21,211,0,MESSANGES,Messanges,Messanges,2118,NaN


In [34]:
result=pd.concat([regions.loc[[10]],departements.loc[[3]], communes.loc[[7563]]],  ignore_index=True, sort=False)

In [36]:
communes.loc[[7563]].append(departements.loc[3], ignore_index=True, sort=False)

,typecom,com,reg,dep,arr,tncc,ncc,nccenr,libelle,can,comparent,cheflieu
0,COM,21407,27.0,21,211,0,MESSANGES,Messanges,Messanges,2118,NaN,NaN
1,NaN,NaN,93.0,04,NaN,4,ALPES DE HAUTE PROVENCE,Alpes-de-Haute-Provence,Alpes-de-Haute-Provence,NaN,NaN,04070


In [42]:
df=pd.DataFrame({'ncc':['France']})

In [43]:
df

,ncc
0,France


In [44]:
df.append(communes.loc[[7563]])

,ncc,typecom,com,reg,dep,arr,tncc,nccenr,libelle,can,comparent
0,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7563,MESSANGES,COM,21407,27.0,21,211,0.0,Messanges,Messanges,2118,NaN


### Essais avec subwords

In [22]:
resultats=[]
for loc in all_locs_ns_str:
    for ref in list(communes_names)+list(departements_names)+list(regions_names):
        if ref in loc.split():
            resultats.append((loc,ref))

KeyboardInterrupt: 

In [26]:
resultats[:100]

[('Paris île-de-france', 'Paris'),
 ('Paris île-de-france', 'Paris'),
 ('Chambéry', 'Chambéry'),
 ('chambéry Chambéry', 'Chambéry'),
 ('orléans métropole Orléans', 'Orléans'),
 ('gamarthe Gamarthe', 'Gamarthe'),
 ('Île-de-France', 'Île-de-France'),
 ('Paris Seine Ouest boulogne billancourt cadre-de-vie composteurs-en-pavillon issy-les-moulineaux larbre marnes-la-coquette mdna meudon plan vanves ville-davray',
  'Paris'),
 ('Paris Seine Ouest boulogne billancourt cadre-de-vie composteurs-en-pavillon issy-les-moulineaux larbre marnes-la-coquette mdna meudon plan vanves ville-davray',
  'Paris'),
 ('antibes Antibes', 'Antibes'),
 ('loire atlantique epci de Loire Atlantique', 'Loire'),
 ('île-de-france Île-de-France', 'Île-de-France'),
 ('île-de-france Île-de-France', 'Île-de-France'),
 ('val de loire Centre Val de Loire', 'Val'),
 ('val de loire Centre Val de Loire', 'Val'),
 ('val de loire Centre Val de Loire', 'Loire'),
 ("hérault de l' Hérault hérault", 'Hérault'),
 ("hérault de l' Hér